## 多クラス分類の精度向上を目指そうの回

### ライブラリのインポート

In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from statsbombpy import sb 
from mplsoccer import Pitch 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pycaret
from pycaret.classification import *
from imblearn.over_sampling import SMOTENC

#import mlflow
#import dagshub

In [3]:
from ydata_profiling import ProfileReport


### 試合のデータ取得

In [2]:
comp = sb.competitions()
#大会一覧取得(competition_id, season_idを知りたい)

In [ ]:
comp #Data Wranger(拡張機能で見ると楽かも？？)

In [2]:
#大会の情報取得
matches = sb.matches(competition_id=72, season_id= 107)


In [3]:
matches #Data Wrangerで(以下略)

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3904629,2023-08-16,13:00:00.000,International - Women's World Cup,2023,Australia Women's,England Women's,1,3,available,...,2023-08-30T11:17:47.551826,6,Semi-finals,Accor Stadium,Tori Penso,Tony Gustavsson,Sarina Glotzbach-Wiegman,1.1.0,2,2
1,3906390,2023-08-20,13:00:00.000,International - Women's World Cup,2023,Spain Women's,England Women's,1,0,available,...,2023-08-22T19:38:43.965521,7,Final,Accor Stadium,Tori Penso,Jorge Vilda,Sarina Glotzbach-Wiegman,1.1.0,2,2
2,3906389,2023-08-19,11:00:00.000,International - Women's World Cup,2023,Sweden Women's,Australia Women's,2,0,available,...,2023-08-29T21:19:57.035390,7,3rd Place Final,Brisbane Stadium,Cheryl Foster,Peter Gerhardsson,Tony Gustavsson,1.1.0,2,2
3,3904628,2023-08-15,11:00:00.000,International - Women's World Cup,2023,Spain Women's,Sweden Women's,2,1,available,...,2023-08-16T11:30:31.556462,6,Semi-finals,Eden Park,Edina Alves Batista,Jorge Vilda,Peter Gerhardsson,1.1.0,2,2
4,3893806,2023-07-26,10:30:00.000,International - Women's World Cup,2023,Spain Women's,Zambia Women's,5,0,available,...,2023-08-15T02:58:57.266594,2,Group Stage,Eden Park,Hyeon-Jeong Oh,Jorge Vilda,Bruce Mwape,1.1.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,3893800,2023-07-24,09:00:00.000,International - Women's World Cup,2023,Italy Women's,Argentina Women's,1,0,available,...,2023-08-12T17:49:18.347307,1,Group Stage,Eden Park,Melissa Paola Borjas Pastrana,Milena Bertolini,Germán Darío Portanova,1.1.0,2,2
60,3893799,2023-07-24,11:30:00.000,International - Women's World Cup,2023,Germany Women's,Morocco Women's,6,0,available,...,2023-08-21T14:33:45.142288,1,Group Stage,AAMI Park,Tori Penso,Martina Voss-Tecklenburg,Reynald Pedros,1.1.0,2,2
61,3893796,2023-07-23,08:00:00.000,International - Women's World Cup,2023,Sweden Women's,South Africa Women's,2,1,available,...,2023-07-30T08:06:34.006738,1,Group Stage,Wellington Regional Stadium,Katja Koroleva,Peter Gerhardsson,Desiree Ellis,1.1.0,2,2
62,3893795,2023-07-22,15:00:00.000,International - Women's World Cup,2023,Denmark Women's,China PR Women's,1,0,available,...,2023-08-16T06:06:04.857484,1,Group Stage,HBF Park,Marie-Soleil Beaudoin,Lars Søndergaard,Qingxia Shui,1.1.0,2,2


In [3]:
ids = list(matches['match_id'])

#print(ids)

In [4]:
#試合毎のイベントデータを一つのDFにまとめる関数
def match_concat(match_all):
    df_event = pd.DataFrame()
    for i in match_all:
        df_i = sb.events(match_id = i) #イベントデータ取得
        df_event = pd.concat([df_event,df_i], ignore_index= True) #くっつける
    return df_event

In [5]:
matches_WWC_2023 = match_concat(match_all = ids)

In [11]:
#GPUでの実行じゃないと時間かかる

ProfileReport_matches_WWC_2023 = ProfileReport(matches_WWC_2023)
ProfileReport_matches_WWC_2023.to_file("ProfileReport_matches_WWC_2023.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

: 

: 

In [ ]:
matches_WWC_2023

In [6]:
event = matches_WWC_2023

#eventをpycaretに認識しやすい形　+　SMOTEで不均衡データのオーバーサンプリングを実施できるようにする
#DataFrameの複製
df2 = event.copy()

#データの抽出(ball_receipt_outcome)

df2 = df2[df2['type'] == 'Ball Receipt*']

df2['ball_receipt_outcome'] = df2['ball_receipt_outcome'].fillna('Complete')


# 結果の確認
#print(df2)

event['ball_receipt_outcome'] = df2['ball_receipt_outcome']

event['ball_receipt_outcome'] = event.ball_receipt_outcome.fillna('0').astype('category')
event['ball_receipt_outcome'] = event.ball_receipt_outcome.fillna('0').astype('category')
event.ball_recovery_recovery_failure = event.ball_recovery_recovery_failure.fillna(False)
event['block_deflection'] = event['block_deflection'].fillna(False)
event['clearance_aerial_won'] = event['clearance_aerial_won'].fillna(False)#.astype('boolean')
event['clearance_head'] = event['clearance_head'].fillna(False)#.astype('boolean')
event['clearance_right_foot'] = event['clearance_right_foot'].fillna(False)#.astype('boolean')
event['clearance_left_foot'] = event['clearance_left_foot'].fillna(False)#.astype('boolean')
event['clearance_body_part'] = event['clearance_body_part'].fillna('0').astype('category')
event['dribble_overrun'] = event['dribble_overrun'].fillna(False)
event['foul_committed_advantage'] = event['foul_committed_advantage'].fillna(False)#.astype('boolean')
event['foul_committed_card'] = event['foul_committed_card'].fillna('No Card')
event['foul_committed_card'] = event['foul_committed_card'].fillna('0').astype('category') 
event['foul_committed_offensive'] = event['foul_committed_offensive'].fillna(False)
event['foul_won_advantage'] = event['foul_won_advantage'].fillna(False)#.astype('boolean')
event['foul_won_defensive'] = event['foul_won_defensive'].fillna(False)
#goalkeepr_end_potition
df2 = event[event['type'] == 'Goal Keeper' ]

event_X_Y= df2['goalkeeper_end_location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#event_X_Ydf

#x座標を代入
df2['goalkeeper_end_location_x'] = event_X_Ydf[0]

#y座標の代入
df2['goalkeeper_end_location_y'] = event_X_Ydf[1]

#元のdataに結合

event['goalkeeper_end_location_x'] = df2['goalkeeper_end_location_x']
event['goalkeeper_end_location_y'] = df2['goalkeeper_end_location_y']

#carry_end_location
df2 = event[event['type'] == 'Carry']

event_X_Y= df2['carry_end_location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#event_X_Ydf

#x座標を代入
df2['carry_end_location_x'] = event_X_Ydf[0]

#y座標の代入
df2['carry_end_location_y'] = event_X_Ydf[1]

#元のdataに結合

event['carry_end_location_x'] = df2['carry_end_location_x']
event['carry_end_location_y'] = df2['carry_end_location_y']

#location
event_X_Y= event['location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#event_X_Ydf

#x座標を代入
event['location_x'] = event_X_Ydf[0]

#y座標の代入
event['location_y'] = event_X_Ydf[1]

event['off_camera'] = event['off_camera'].fillna(False)
event['out'] = event['out'].fillna(False)
event['counterpress'] = event['counterpress'].fillna(False)
event['pass_aerial_won'] = event['pass_aerial_won'].fillna(False)#.astype('boolean')
event['pass_cross'] = event['pass_cross'].fillna(False)#.astype('boolean')
event['pass_cut_back'] = event['pass_cut_back'].fillna(False)#.astype('boolean')
event['pass_goal_assist'] = event['pass_goal_assist'].fillna(False)#.astype('boolean')
event['pass_inswinging'] = event['pass_inswinging'].fillna(False)#.astype('boolean')
event['pass_no_touch'] = event['pass_no_touch'].fillna(False)#.astype('boolean')
event['pass_outswinging'] = event['pass_outswinging'].fillna(False)#.astype('boolean')
event['pass_through_ball'] = event['pass_through_ball'].fillna(False)#.astype('boolean')
event['pass_shot_assist'] = event['pass_shot_assist'].fillna(False)#.astype('boolean')
event['pass_straight'] = event['pass_straight'].fillna(False)#.astype('boolean')
event['pass_switch'] = event['pass_switch'].fillna(False)


event_X_Y= event['pass_end_location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#event_X_Ydf

#x座標を代入
event['pass_end_location_x'] = event_X_Ydf[0]

#y座標の代入
event['pass_end_location_y'] = event_X_Ydf[1]


event['shot_aerial_won'] = event['shot_aerial_won'].fillna(False)#.astype('boolean')
event['shot_first_time'] = event['shot_first_time'].fillna(False)#.astype('boolean')
event['shot_one_on_one'] = event['shot_one_on_one'].fillna(False)
event['under_pressure'] = event['under_pressure'].fillna(False)

#shot_end_location
event_X_Y= event['shot_end_location']
event_X_Ydf =event_X_Y.apply(pd.Series)

#x座標を代入
event['shot_end_location_x'] = event_X_Ydf[0]

#y座標の代入
event['shot_end_location_y'] = event_X_Ydf[1]

#z座標の取得
event['shot_end_location_z'] = event_X_Ydf[2]

#元のdataに結合

#event['goalkeeper_end_location_x'] = df2['goalkeeper_end_location_x']
#event['goalkeeper_end_location_y'] = df2['goalkeeper_end_location_y']

event.dribble_outcome = event.dribble_outcome.fillna('0').astype('category')
event.duel_outcome = event.duel_outcome.fillna('0').astype('category')
event.duel_type =event.duel_type.fillna('0').astype('category')
event.duration = event.duration.fillna('0')#.astype('float')
event.foul_committed_type = event.foul_committed_type.fillna('0').astype('category')
event.goalkeeper_body_part = event.goalkeeper_body_part.fillna('0').astype('category')
event.goalkeeper_outcome = event.goalkeeper_outcome.fillna('0').astype('category')
event.goalkeeper_position = event.goalkeeper_position.fillna('0').astype('category')
event.goalkeeper_technique = event.goalkeeper_technique.fillna('0').astype('category')
event.goalkeeper_type = event.goalkeeper_type.fillna('0').astype('category')
event.interception_outcome = event.interception_outcome.fillna('0').astype('category')
event.pass_assisted_shot_id = event.pass_assisted_shot_id.fillna('0').astype('category')
event.pass_body_part = event.pass_body_part.fillna('0').astype('category')
event.pass_height = event.pass_height.fillna('0').astype('category')
event.pass_length = event.pass_length.fillna('0').astype('category')
event.pass_outcome = event.pass_outcome.fillna('0').astype('category')
event.pass_recipient = event.pass_recipient.fillna('0').astype('category')
event.pass_recipient_id = event.pass_recipient_id.fillna('0').astype('category')
event.pass_technique = event.pass_technique.fillna('0').astype('category')
event.pass_type = event.pass_type.fillna('0').astype('category')
event.player = event.player.fillna('No player').astype('category')
event.player_id = event.player_id.fillna('0').astype('int')
event.position = event.position.fillna('0').astype('category')
event.shot_body_part = event.shot_body_part.fillna('0').astype('category')
event.shot_key_pass_id = event.shot_key_pass_id.fillna('0').astype('category')
event.shot_outcome = event.shot_outcome.fillna('0').astype('category')
event.shot_statsbomb_xg = event.shot_statsbomb_xg.fillna('0.0').astype('float')
event.shot_technique = event.shot_technique.fillna('0').astype('category')
event.substitution_replacement = event.substitution_replacement.fillna('0').astype('category')
event.substitution_replacement_id = event.substitution_replacement_id.fillna('0').astype('int')
event.play_pattern = event.play_pattern.astype('category')
event.id = event.id.astype('category')
event.possession_team = event.possession_team.astype('category')
event.team = event.team.astype('category')



event = event.drop(['shot_type', 'substitution_outcome_id','substitution_outcome',
                    'related_events', 'tactics', 'shot_freeze_frame','shot_end_location',
                    'pass_end_location','injury_stoppage_in_chain',
                    'goalkeeper_end_location', 'carry_end_location','location','50_50'],
                    axis=1)


#event

In [8]:
#実験用なので実行しない
event_2 = event.copy()

event_2['end_location_x'] =event_2[['pass_end_location_x', 'carry_end_location_x', 'shot_end_location_x']].iloc[:,0]
event_2['end_location_y'] =event_2[['pass_end_location_y', 'carry_end_location_y', 'shot_end_location_y']].iloc[:,0]

,bad_behaviour_card,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,counterpress,...,carry_end_location_x,carry_end_location_y,location_x,location_y,pass_end_location_x,pass_end_location_y,shot_end_location_x,shot_end_location_y,shot_end_location_z,end_location_x
0,NaN,0,False,False,False,0,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0,False,False,False,0,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0,False,False,False,0,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0,False,False,False,0,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0,False,False,False,0,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226141,NaN,0,False,False,False,0,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226142,NaN,0,False,False,False,0,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226143,NaN,0,False,False,False,0,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
226144,NaN,0,False,False,False,0,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
event['end_location_x'] =event[['pass_end_location_x', 'carry_end_location_x', 'shot_end_location_x']].iloc[:,0]
event['end_location_y'] =event[['pass_end_location_y', 'carry_end_location_y', 'shot_end_location_y']].iloc[:,0]

event.bad_behaviour_card = event.bad_behaviour_card.fillna('0').astype('category')
event.dribble_nutmeg = event.dribble_nutmeg.fillna(False)
event.ball_recovery_offensive = event.ball_recovery_offensive.fillna(False)
event.clearance_other = event.clearance_other.fillna(False)
event.foul_committed_penalty = event.foul_committed_penalty.fillna(False)
event.miscontrol_aerial_won = event.miscontrol_aerial_won.fillna(False)
event.pass_deflected = event.pass_deflected.fillna(False)
event.shot_deflected = event.shot_deflected.fillna(False)
event.foul_won_penalty = event.foul_won_penalty.fillna(False)
event.block_offensive = event.block_offensive.fillna(False)
event.pass_miscommunication = event.pass_miscommunication.fillna(False)
event.goalkeeper_lost_in_play = event.goalkeeper_lost_in_play.fillna(False)
event.goalkeeper_shot_saved_to_post = event.goalkeeper_shot_saved_to_post.fillna(False)
event.shot_open_goal = event.shot_open_goal.fillna(False)
event.shot_saved_to_post = event.shot_saved_to_post.fillna(False)
event.goalkeeper_shot_saved_off_target = event.goalkeeper_shot_saved_off_target.fillna(False)
event.goalkeeper_success_in_play = event.goalkeeper_success_in_play.fillna(False)
event.shot_saved_off_target = event.shot_saved_off_target.fillna(False)
event.goalkeeper_punched_out = event.goalkeeper_punched_out.fillna(False)
event.block_save_block = event.block_save_block.fillna(False)
event.goalkeeper_penalty_saved_to_post = event.goalkeeper_penalty_saved_to_post.fillna(False)
event.dribble_no_touch = event.dribble_no_touch.fillna(False)
event.goalkeeper_success_out = event.goalkeeper_success_out.fillna(False)
event.shot_follows_dribble = event.shot_follows_dribble.fillna(False)
event.player_off_permanent = event.player_off_permanent.fillna(False)
event.shot_redirect = event.shot_redirect.fillna(False)



In [10]:
event.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226146 entries, 0 to 226145
Data columns (total 116 columns):
 #    Column                            Dtype   
---   ------                            -----   
 0    bad_behaviour_card                category
 1    ball_receipt_outcome              category
 2    ball_recovery_recovery_failure    bool    
 3    block_deflection                  bool    
 4    clearance_aerial_won              bool    
 5    clearance_body_part               category
 6    clearance_head                    bool    
 7    clearance_left_foot               bool    
 8    clearance_right_foot              bool    
 9    counterpress                      bool    
 10   dribble_nutmeg                    bool    
 11   dribble_outcome                   category
 12   duel_outcome                      category
 13   duel_type                         category
 14   duration                          object  
 15   foul_committed_advantage          bool    
 16   

In [ ]:
event

In [10]:
event_fi.value_counts('type')

type
Others     117371
Pass        59837
Dribble     48938
Name: count, dtype: int64

In [8]:
event_fi = event.drop(['carry_end_location_x', 'carry_end_location_y', 'goalkeeper_end_location_x', 
                       'goalkeeper_end_location_y', 'pass_end_location_x', 'pass_end_location_y', 
                       'shot_end_location_x', 'shot_end_location_y', 'shot_end_location_z'], axis=1)

event_fi = event_fi.replace({'type':{'Carry':'Dribble'}})

#replace({'age': [24, 18], 'point': 70}, 100))

event_fi = event_fi.replace({'type':
                            ['Ball Receipt*', 'Pressure','Ball Recovery', 'Duel', 'Goal Keeper', 'Shot', 'Block', 'Miscontrol',  'Clearance', 'Dispossessed',
                             'Interception', 'Foul Committed', 'Dribbled Past', 'Foul Won', 'Substitution','Half Start', 'Half End', 'Injury Stoppage',
                             'Player Off','Player On','Referee Ball-Drop','Starting XI', 'Tactical Shift', 'Shield', 'Error','50/50','Offside', 'Bad Behaviour',
                             'Own Goal Against', 'Own Goal For']},
                            'Others')

#event_fi
#event_fi.info()

le = LabelEncoder()
le.fit(event_fi['type'])

event_fi['type_num'] = le.transform(event_fi['type'])

event_fi

,bad_behaviour_card,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,counterpress,...,dribble_no_touch,goalkeeper_success_out,shot_follows_dribble,player_off_permanent,shot_redirect,location_x,location_y,end_location_x,end_location_y,type_num
0,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,1
1,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,1
2,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,1
3,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,1
4,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226141,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,1
226142,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,1
226143,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,1
226144,0,0,False,False,False,0,False,False,False,False,...,False,False,False,False,False,NaN,NaN,NaN,NaN,1


# pycaret用オブジェクトとデータ分割、学習

In [9]:

event_fi_py = event_fi.drop(["type","id", "pass_recipient","pass_recipient_id", "pass_aerial_won", "pass_assisted_shot_id","pass_outcome","pass_angle",
                             "pass_body_part", "pass_height", "pass_inswinging", "pass_length", "pass_type", "player_id", "team_id",'timestamp'], axis=1)

In [47]:
ml_data = event_fi_py.sample(frac=0.8, random_state=5).reset_index(drop=True)
ml_data_unseen = event_fi_py.drop(ml_data.index).reset_index(drop=True)


In [ ]:
#実行はしない(Data Wrenger)

ml_data_unseen

In [10]:
exp_trial = setup(data = event_fi_py, target= 'type_num' ,
                  fix_imbalance = True, 
                  
                  session_id= 56)
#log_experiment = "dagshub", experiment_name = 'trial_classification_2', log_plots = True, log_data =True,use_gpu=True

,Description,Value
0,Session id,56
1,Target,type_num
2,Target type,Multiclass
3,Original data shape,"(226146, 92)"
4,Transformed data shape,"(314324, 213)"
5,Transformed train set shape,"(246480, 213)"
6,Transformed test set shape,"(67844, 213)"
7,Numeric features,13
8,Categorical features,26
9,Rows with missing values,73.5%


In [14]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9255,0.9864,0.9255,0.9338,0.9268,0.8810,0.8843,49.1050
dt,Decision Tree Classifier,0.8806,0.9133,0.8806,0.9009,0.8834,0.8114,0.8199,60.1660
gbc,Gradient Boosting Classifier,0.8756,0.0000,0.8756,0.8943,0.8586,0.7852,0.8070,316.4390
et,Extra Trees Classifier,0.8383,0.9629,0.8383,0.8321,0.8216,0.7234,0.7368,74.8980
rf,Random Forest Classifier,0.8144,0.9876,0.8144,0.8546,0.7604,0.6677,0.7206,65.0450
qda,Quadratic Discriminant Analysis,0.8046,0.0000,0.8046,0.8775,0.8056,0.7087,0.7450,42.7540
xgboost,Extreme Gradient Boosting,0.8013,0.9666,0.8013,0.8410,0.7362,0.6425,0.7003,66.3460
ada,Ada Boost Classifier,0.7918,0.0000,0.7918,0.8215,0.7154,0.6227,0.6885,71.0430
lda,Linear Discriminant Analysis,0.6732,0.0000,0.6732,0.7451,0.6841,0.4989,0.5211,59.1180
ridge,Ridge Classifier,0.6696,0.0000,0.6696,0.7433,0.6796,0.4937,0.5175,52.1300


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=56, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [11]:
lightgbm = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7814,0.8429,0.7814,0.8175,0.7765,0.6481,0.6657
1,0.9391,0.9877,0.9391,0.9423,0.9398,0.9021,0.9031
2,0.9270,0.9845,0.9270,0.9319,0.9281,0.8830,0.8846
3,0.9372,0.9877,0.9372,0.9378,0.9374,0.8982,0.8983
4,0.9268,0.9862,0.9268,0.9373,0.9286,0.8838,0.8878
5,0.9210,0.9880,0.9210,0.9367,0.9233,0.8754,0.8818
6,0.9138,0.9881,0.9138,0.9353,0.9168,0.8650,0.8740
7,0.9078,0.9858,0.9078,0.9314,0.9110,0.8558,0.8659
8,0.9303,0.9857,0.9303,0.9346,0.9313,0.8881,0.8895


In [22]:
evaluate_model(lightgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [12]:
tuned_lightgbm = tune_model(lightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9399,0.9932,0.9399,0.9414,0.9377,0.9001,0.9029
1,0.9522,0.9938,0.9522,0.9521,0.9521,0.9223,0.9223
2,0.9515,0.9933,0.9515,0.9526,0.9518,0.9216,0.9219
3,0.9565,0.9943,0.9565,0.9560,0.9561,0.9288,0.9290
4,0.9563,0.9943,0.9563,0.9560,0.9561,0.9288,0.9288
5,0.9630,0.9955,0.9630,0.9641,0.9633,0.9401,0.9405
6,0.9495,0.9946,0.9495,0.9534,0.9503,0.9188,0.9202
7,0.9354,0.9940,0.9354,0.9447,0.9369,0.8974,0.9010
8,0.9553,0.9952,0.9553,0.9565,0.9556,0.9278,0.9282


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [14]:
get_config()

{'USI',
 'X',
 'X_test',
 'X_test_transformed',
 'X_train',
 'X_train_transformed',
 'X_transformed',
 '_available_plots',
 '_ml_usecase',
 'data',
 'dataset',
 'dataset_transformed',
 'exp_id',
 'exp_name_log',
 'fix_imbalance',
 'fold_generator',
 'fold_groups_param',
 'fold_shuffle_param',
 'gpu_n_jobs_param',
 'gpu_param',
 'html_param',
 'idx',
 'is_multiclass',
 'log_plots_param',
 'logging_param',
 'memory',
 'n_jobs_param',
 'pipeline',
 'seed',
 'target_param',
 'test',
 'test_transformed',
 'train',
 'train_transformed',
 'variable_and_property_keys',
 'variables',
 'y',
 'y_test',
 'y_test_transformed',
 'y_train',
 'y_train_transformed',
 'y_transformed'}

In [24]:
evaluate_model(tuned_lightgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [50]:
rf = create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7413,0.9224,0.7413,0.7951,0.6900,0.5678,0.6035
1,0.7315,0.9146,0.7315,0.8145,0.6881,0.5619,0.6085
2,0.7386,0.9149,0.7386,0.8083,0.6886,0.5677,0.6086
3,0.7380,0.9166,0.7380,0.8114,0.6887,0.5699,0.6154
4,0.7327,0.9143,0.7327,0.8069,0.6846,0.5604,0.6035
5,0.7435,0.9211,0.7435,0.8099,0.6962,0.5769,0.6179
6,0.7352,0.9163,0.7352,0.8121,0.6887,0.5660,0.6111
7,0.7404,0.9174,0.7404,0.8033,0.6899,0.5709,0.6122
8,0.7370,0.9168,0.7370,0.8135,0.6910,0.5685,0.6125


In [54]:
tuned_rf = tune_model(rf, optimize='Recall')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6834,0.8824,0.6834,0.8217,0.6537,0.5000,0.5667
1,0.6856,0.8896,0.6856,0.8228,0.6550,0.5031,0.5699
2,0.6864,0.8885,0.6864,0.8243,0.6546,0.5043,0.5718
3,0.6846,0.8834,0.6846,0.8165,0.6509,0.5012,0.5686
4,0.6807,0.8839,0.6807,0.8240,0.6488,0.4960,0.5644
5,0.6888,0.8733,0.6888,0.8224,0.6586,0.5077,0.5731
6,0.6867,0.8848,0.6867,0.8191,0.6544,0.5043,0.5705
7,0.6854,0.8745,0.6854,0.8185,0.6526,0.5026,0.5698
8,0.6882,0.8660,0.6882,0.8234,0.6568,0.5065,0.5722


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [55]:
tuned_rf_prec = tune_model(rf, optimize='Prec.')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6834,0.8824,0.6834,0.8217,0.6537,0.5000,0.5667
1,0.6856,0.8896,0.6856,0.8228,0.6550,0.5031,0.5699
2,0.6864,0.8885,0.6864,0.8243,0.6546,0.5043,0.5718
3,0.6846,0.8834,0.6846,0.8165,0.6509,0.5012,0.5686
4,0.6807,0.8839,0.6807,0.8240,0.6488,0.4960,0.5644
5,0.6888,0.8733,0.6888,0.8224,0.6586,0.5077,0.5731
6,0.6867,0.8848,0.6867,0.8191,0.6544,0.5043,0.5705
7,0.6854,0.8745,0.6854,0.8185,0.6526,0.5026,0.5698
8,0.6882,0.8660,0.6882,0.8234,0.6568,0.5065,0.5722


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [56]:
tuned_rf_F1 = tune_model(rf, optimize='F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6834,0.8824,0.6834,0.8217,0.6537,0.5000,0.5667
1,0.6856,0.8896,0.6856,0.8228,0.6550,0.5031,0.5699
2,0.6864,0.8885,0.6864,0.8243,0.6546,0.5043,0.5718
3,0.6846,0.8834,0.6846,0.8165,0.6509,0.5012,0.5686
4,0.6807,0.8839,0.6807,0.8240,0.6488,0.4960,0.5644
5,0.6888,0.8733,0.6888,0.8224,0.6586,0.5077,0.5731
6,0.6867,0.8848,0.6867,0.8191,0.6544,0.5043,0.5705
7,0.6854,0.8745,0.6854,0.8185,0.6526,0.5026,0.5698
8,0.6882,0.8660,0.6882,0.8234,0.6568,0.5065,0.5722


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [57]:
tuned_rf_Kappa = tune_model(rf, optimize='Kappa')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6834,0.8824,0.6834,0.8217,0.6537,0.5000,0.5667
1,0.6856,0.8896,0.6856,0.8228,0.6550,0.5031,0.5699
2,0.6864,0.8885,0.6864,0.8243,0.6546,0.5043,0.5718
3,0.6846,0.8834,0.6846,0.8165,0.6509,0.5012,0.5686
4,0.6807,0.8839,0.6807,0.8240,0.6488,0.4960,0.5644
5,0.6888,0.8733,0.6888,0.8224,0.6586,0.5077,0.5731
6,0.6867,0.8848,0.6867,0.8191,0.6544,0.5043,0.5705
7,0.6854,0.8745,0.6854,0.8185,0.6526,0.5026,0.5698
8,0.6882,0.8660,0.6882,0.8234,0.6568,0.5065,0.5722


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [58]:
tuned_rf_MCC = tune_model(rf, optimize='MCC')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6834,0.8824,0.6834,0.8217,0.6537,0.5000,0.5667
1,0.6856,0.8896,0.6856,0.8228,0.6550,0.5031,0.5699
2,0.6864,0.8885,0.6864,0.8243,0.6546,0.5043,0.5718
3,0.6846,0.8834,0.6846,0.8165,0.6509,0.5012,0.5686
4,0.6807,0.8839,0.6807,0.8240,0.6488,0.4960,0.5644
5,0.6888,0.8733,0.6888,0.8224,0.6586,0.5077,0.5731
6,0.6867,0.8848,0.6867,0.8191,0.6544,0.5043,0.5705
7,0.6854,0.8745,0.6854,0.8185,0.6526,0.5026,0.5698
8,0.6882,0.8660,0.6882,0.8234,0.6568,0.5065,0.5722


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [59]:
tuned_rf_ac = tune_model(rf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6834,0.8824,0.6834,0.8217,0.6537,0.5000,0.5667
1,0.6856,0.8896,0.6856,0.8228,0.6550,0.5031,0.5699
2,0.6864,0.8885,0.6864,0.8243,0.6546,0.5043,0.5718
3,0.6846,0.8834,0.6846,0.8165,0.6509,0.5012,0.5686
4,0.6807,0.8839,0.6807,0.8240,0.6488,0.4960,0.5644
5,0.6888,0.8733,0.6888,0.8224,0.6586,0.5077,0.5731
6,0.6867,0.8848,0.6867,0.8191,0.6544,0.5043,0.5705
7,0.6854,0.8745,0.6854,0.8185,0.6526,0.5026,0.5698
8,0.6882,0.8660,0.6882,0.8234,0.6568,0.5065,0.5722


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


### pycaretのデータ分析

In [16]:
dataset_transformed =get_config('dataset_transformed')



,bad_behaviour_card_0,bad_behaviour_card_Yellow Card,bad_behaviour_card_Red Card,ball_receipt_outcome_0,ball_receipt_outcome_Complete,ball_receipt_outcome_Incomplete,ball_recovery_recovery_failure,block_deflection,clearance_aerial_won,clearance_body_part_0,...,dribble_no_touch,goalkeeper_success_out,shot_follows_dribble,player_off_permanent,shot_redirect,location_x,location_y,end_location_x,end_location_y,type_num
177840,1.0,0.0,0.0,1.0,0.0,0.0,False,False,False,1.0,...,False,False,False,False,False,104.599998,60.700001,114.099998,39.299999,2
131206,1.0,0.0,0.0,1.0,0.0,0.0,False,False,False,1.0,...,False,False,False,False,False,8.900000,7.000000,64.757394,39.873917,0
48879,1.0,0.0,0.0,1.0,0.0,0.0,False,False,False,1.0,...,False,False,False,False,False,52.299999,54.799999,64.757394,39.873917,0
7736,1.0,0.0,0.0,0.0,1.0,0.0,False,False,False,1.0,...,False,False,False,False,False,33.200001,69.699997,64.757394,39.873917,1
37927,1.0,0.0,0.0,1.0,0.0,0.0,False,False,False,1.0,...,False,False,False,False,False,103.699997,22.700001,64.757394,39.873917,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65168,1.0,0.0,0.0,1.0,0.0,0.0,False,False,False,1.0,...,False,False,False,False,False,3.100000,42.400002,64.757394,39.873917,0
72210,1.0,0.0,0.0,1.0,0.0,0.0,False,False,False,1.0,...,False,False,False,False,False,67.000000,18.799999,64.757394,39.873917,0
199445,1.0,0.0,0.0,0.0,1.0,0.0,False,False,False,1.0,...,False,False,False,False,False,91.599998,28.799999,64.757394,39.873917,1
122308,1.0,0.0,0.0,1.0,0.0,0.0,False,False,False,1.0,...,False,False,False,False,False,22.299999,52.799999,31.799999,47.799999,2


In [ ]:
#Dribble(type_num = 0)のデータを抽出
dataset_transformed_dribble = dataset_transformed[dataset_transformed['type_num'] == 0]

#[duration, location_x, loaction_y, end_location_x,  end_location_y,]を抽出
dataset_transformed_dribble = dataset_transformed_dribble[['duration', 'location_x', 'location_y', 'end_location_x', 'end_location_y']]


In [ ]:
#others(type_num = 1)のデータを抽出
dataset_transformed_others = dataset_transformed[dataset_transformed['type_num'] == 1]

#[duration, location_x, loaction_y, end_location_x,  end_location_y,]を抽出
dataset_transformed_others = dataset_transformed_others[['duration', 'location_x', 'location_y', 'end_location_x', 'end_location_y']]

In [ ]:
#pass(type_num = 2)のデータを抽出
dataset_transformed_pass = dataset_transformed[dataset_transformed['type_num'] == 2]

#[duration, location_x, loaction_y, end_location_x,  end_location_y,]を抽出
dataset_transformed_pass = dataset_transformed_pass[['duration', 'location_x', 'location_y', 'end_location_x', 'end_location_y']]
